In [762]:
import pandas as pd
import numpy as np

In [834]:
frames = []
for i in range(4):
    frames.append(pd.read_csv('../data/Sunflower_201'+str(i+2)+'.csv'))

for df in frames:
    df.drop(df.columns[[0,1]],axis=1,inplace=True)

X_ndvi_list = []
Y_list = []

In [835]:
for df in frames:
    X_ndvi_list.append(df.loc[:, df.columns != 'yield'])
    Y_list.append(df.loc[:, df.columns == 'yield'])


In [836]:
for i in range(4):
    X_ndvi_list[i].columns = range(-52,0,1)


In [837]:
X_ndvi_list[1].shape

(24, 52)

In [838]:
frames = []
for i in range(4):
    frames.append(pd.DataFrame.from_csv('../meteo_csv/201'+str(i+2)+'.csv'))


In [839]:
for df in frames:
    df.drop(df.columns[[0, 1,4,6,8,10,12,14,16]], axis=1, inplace=True)
    df['year'] = df['yearmoda'] // 10000
    df['month'] = df['yearmoda'] // 100 % 100
    df['day'] = df['yearmoda'] % 100
    df['sndp'].replace(999.9,0, inplace = True)
    df['yearmoda1'] = df['yearmoda'].astype(str).apply(lambda x: x[:9])
    df.index = pd.to_datetime(df['yearmoda1'], format='%Y%m%d').values
    df.drop(df.columns[[0,16]], axis=1, inplace=True)
    for x in df[df['wdsp'] == 999.9].index:
        df.loc[x,'wdsp'] = np.NaN
    df['wdsp'] = df['wdsp'].interpolate(method='time')
    for x in df[df['mxspd'] == 999.9].index:
        df.loc[x,'mxspd'] = np.NaN
    df['mxspd'] = df['mxspd'].interpolate(method='time')
    for x in df[df['prcp'] == '99.99'].index:
        df.loc[x,'prcp'] = np.NaN
    for x in df.index:
        if (str(df.loc[x,'prcp'])[-1] == 'I') or (str(df.loc[x,'prcp'])[-1] == 'H'):
            df.loc[x,'prcp'] = np.NaN
    for x in df[df['prcp'].notnull()].index:
        df.loc[x,'prcp'] = df.loc[x,'prcp'][:-1]
    for x in df.index:
        if (str(df.loc[x,'max'])[-1] == '*'):
            df.loc[x,'max'] = df.loc[x,'max'][:-1]
        if (str(df.loc[x,'min'])[-1] == '*'):
            df.loc[x,'min'] = df.loc[x,'min'][:-1]
    df['max']=df['max'].astype(np.float64)
    df['min']=df['min'].astype(np.float64)
    df['prcp']=df['prcp'].astype(np.float64)
    df['prcp'] = df['prcp'].interpolate(method='time')
    
    
    


In [840]:
for df in frames:
    df['fog'] = df['frshtt'] // 100000
    df['rain'] = df['frshtt'] // 10000 % 10
    df['snow'] = df['frshtt'] // 1000 % 10
    df['hail'] = df['frshtt'] // 100 % 10
    df['thunder'] = df['frshtt'] // 10 % 10
    df['tornado'] = df['frshtt'] % 10
    df.drop(['frshtt'],axis = 1, inplace = True)


    
    
    

In [841]:
frames[0].shape

(366, 20)

In [842]:
for i in range(3):
    frames[i] = frames[i].fillna(method='bfill')

In [843]:
years = []
years.append(pd.Series(frames[0].values.ravel()[:-40]))
years.append(pd.Series(frames[1].values.ravel()))
years.append(pd.Series(frames[2].values.ravel()))
years.append(pd.Series(frames[3].values.ravel()[:-20]))
for year in years:
    print(year.shape)

(7280,)
(7280,)
(7280,)
(7280,)


In [844]:
years[1]

0         28.40
1         25.00
2       1022.50
3        999.40
4          7.40
5          5.30
6          7.80
7         35.60
8         19.90
9          0.13
10         3.90
11      2013.00
12         1.00
13         1.00
14         1.00
15         0.00
16         0.00
17         0.00
18         0.00
19         0.00
20        32.40
21        30.80
22      1022.80
23       999.90
24         3.30
25         6.80
26         9.70
27        36.70
28        26.20
29         0.13
         ...   
7250       0.00
7251    2013.00
7252      12.00
7253      30.00
7254       1.00
7255       1.00
7256       0.00
7257       0.00
7258       0.00
7259       0.00
7260      32.20
7261      31.70
7262    1030.20
7263    1007.10
7264       1.20
7265       6.80
7266       9.70
7267      34.30
7268      28.80
7269       0.02
7270       0.00
7271    2013.00
7272      12.00
7273      31.00
7274       1.00
7275       0.00
7276       0.00
7277       0.00
7278       0.00
7279       0.00
Length: 7280, dtype: flo

In [845]:
for i in range(4):
    X_ndvi_list[i] = X_ndvi_list[i].transpose()
    print(X_ndvi_list[i].shape)


(52, 26)
(52, 24)
(52, 54)
(52, 44)


In [846]:
for i in range(4):

    df = pd.DataFrame(years[i])
    for j in range(len(Y_list[i].index)-1):
        df = pd.concat([df,years[i]], axis=1)
    df.columns = range(len(Y_list[i].index))
    X1[i] = pd.concat([X_ndvi_list[i],df], join = 'inner')
    X1[i] = X1[i].transpose()
    print(X1[i].shape)

(26, 7332)
(24, 7332)
(54, 7332)
(44, 7332)


In [847]:
X_train, X_test, Y_train, Y_test = pd.concat([X1[0],X1[1],X1[2]]), X1[3], pd.concat([Y_list[0],Y_list[1],Y_list[2]]), Y_list[3] 

In [848]:
X_train.index = range(104)
Y_train.index = range(104)


In [849]:
Y_train = pd.Series(Y_train.values.ravel())
Y_test = pd.Series(Y_test.values.ravel())

# X_tr = X_train.values
# Y_tr = Y_train.values
# X_tst = X_test.values
# Y_tst = Y_test.values

# print('X_tr', X_tr.shape)
# print('X_tst', X_tst.shape)
# print('Y_tr', Y_tr.shape)
# print('Y_tst', Y_tst.shape)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(104, 7332)
(44, 7332)
(104,)
(44,)


In [850]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [851]:
train_data = pd.concat([X_train, Y_train], axis=1)
test_data = pd.concat([X_test, Y_test], axis=1)

In [852]:
data = pd.concat([train_data, test_data])
data.shape

(148, 7333)

In [853]:
scaler = MinMaxScaler()
data.iloc[:,:data.shape[1] - 1] = scaler.fit_transform(data.iloc[:,:data.shape[1] - 1])

In [854]:
data = data.dropna()

In [855]:
data.head()

,-52,-51,-50,-49,-48,-47,-46,-45,-44,-43,...,7271,7272,7273,7274,7275,7276,7277,7278,7279,0
0,0.134078,0.142012,0.141355,0.140704,0.140058,0.124155,0.123581,0.123013,0.122449,0.123167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.70
1,0.287709,0.304734,0.303324,0.301926,0.300542,0.286740,0.285414,0.284100,0.282799,0.284457,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75
2,0.156425,0.165680,0.164914,0.164154,0.163401,0.147804,0.147121,0.146444,0.145773,0.146628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.88
3,0.198324,0.210059,0.209087,0.208124,0.207170,0.192145,0.191257,0.190377,0.189504,0.190616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.69
4,0.027933,0.029586,0.029449,0.029313,0.029179,0.011824,0.011770,0.011715,0.011662,0.011730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.83


In [856]:
train_data, test_data = train_test_split(data)

print(train_data.shape)
print(test_data.shape)

(109, 7333)
(37, 7333)


In [857]:
n = train_data.shape[1]

X_train = train_data.iloc[:,:(n-1)]
X_test = test_data.iloc[:,:(n-1)]
Y_train = train_data.iloc[:,(n-1)]
Y_test = test_data.iloc[:,(n-1)]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(109, 7332)
(37, 7332)
(109,)
(37,)


In [858]:
#X_sm_train = X_train.drop(X_train.columns[range(200,-1)], axis=1)

In [862]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred)

0.27361798044644381

2

In [687]:
X_train.iloc[100,500]

4.2999999999999998

In [832]:
np.isfinite(Y_train).sum().sum()

109

In [827]:
X_train.shape[0] * X_train.shape[1]

813852

In [829]:
for dt in X_train.dtypes:
    if dt != np.float64:
        print(type(dt))

In [833]:
Y_train.unique()

array([ 0.94,  1.67,  1.13,  1.89,  1.61,  2.86,  1.37,  2.49,  0.97,
        1.46,  1.95,  2.42,  1.2 ,  2.85,  3.01,  1.4 ,  2.52,  2.22,
        1.47,  2.4 ,  2.16,  1.11,  0.7 ,  3.02,  0.84,  2.1 ,  1.58,
        2.9 ,  1.57,  1.77,  1.72,  1.43,  1.98,  3.13,  1.28,  0.76,
        0.98,  0.81,  3.44,  1.86,  2.2 ,  1.29,   nan,  1.75,  2.57,
        1.87,  1.9 ,  3.15,  0.13,  2.39,  2.97,  2.51,  1.55,  2.17,
        1.79,  2.99,  2.5 ,  1.15,  1.91,  3.09,  2.04,  1.62,  1.26,
        3.31,  0.95,  2.58,  0.88,  0.96,  1.73,  2.94,  3.22,  1.94,
        2.76,  2.68,  2.3 ,  2.78,  2.83,  2.89,  2.7 ,  0.77,  2.37,
        3.05,  1.18,  0.9 ,  3.23,  0.75,  0.65,  1.05])